# Ejercicio 1

En este ejercicio el objetivo es tener nuestra primera versión de un clasificador de imágenes. Particularmente, el objetivo es distinguir si una foto es de un hot-dog o no. Es necesario tener instalado

- NumPy.
- OpenCV.

Para no tener que instalar todo localmente y tener problemas de compatibilidad, se puede utilizar gratuitamente Google Colab. Esto nos da disponibilidad de recursos computacionales y un ambiente con muchas librerías ya instaladas.

## Conseguir dataset

La siguiente celda se encarga de importar algunas librerías que usaremos en el cuaderno. 
  - OpenCV es una librería de visión por computadora. Tiene diferentes algoritmos que permiten trabajar con imágenes, así como aplicar diferentes técnicas de visión por computadora.
  - NumPy es una librería de Python que permite trabajar con matrices de gran tamaño de manera eficiente.

In [0]:
import glob
import  cv2
import numpy as np

Colab, el ambiente que estaremos utilizando, no sólo tiene las herramientas habituales de cuadernos de Python (archivos .ipynb). Entre sus ventajas, viene con la capacidad de poder utilizar `pip install` para instalar librerías adicionales a las que ya vienen en su ambiente. Ejecuta la siguiente celda para instalar la librería google_images_download, la cuál permite descargar imágenes de Google Image Search.

In [0]:
!pip install google_images_download

     |████████████████████████████████| 911kB 7.0MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14550 sha256=9616149c7901752f5d91416db7317252e0f0adc0ce0442332eb247d08f84c61e
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


**Ejercicio 1**. En la siguiente celda, implementa código para descargar imágenes de hamburguesas y de hot dogs. El objetivo es descargarlas en dos carpetas diferentes. La estructura del output debe ser:

```
- downloads
-- hamburguer
--- foto1.jpg
--- ...
-- hot-dog
--- foto1.jpg
--- ...
```

Intenta descargar por lo menos 90 fotos. En el panel de la izquierda, puedes revisar las fotos descargadas en la pestaña llamada "Files".

In [0]:
from google_images_download import google_images_download 
response = google_images_download.googleimagesdownload()

arguments = {"keywords":"hot-dog","limit":10,"o":"hot-dog"} 
paths = response.download(arguments)

arguments = {"keywords":"hamburger","limit":10,"o":"burger"} 
paths = response.download(arguments)


Item no.: 1 --> Item name = hot-dog
Evaluating...
Starting Download...
Completed Image ====> 1.1200px-Hotdog_-_Evan_Swigart.jpg
Completed Image ====> 2.Basic-Hot-Dogs2-600x500.jpg
Completed Image ====> 3.giada-hot-dog-today-tease-171013_cc8ea08d065ebe92133af1c5cd1c868e.jpg
Completed Image ====> 4.Hero_Dog_0.png
Completed Image ====> 5.52394_3000x2000.jpg
Completed Image ====> 6.Seattle_Dog.jpg
Completed Image ====> 7.best-hot-dog-fp.jpg
Completed Image ====> 8.hotdog_0.png
Completed Image ====> 9.hotdog-royalty-free-image-185123377-1562609410.jpg
Completed Image ====> 10.1535214434384.jpeg

Errors: 0


Item no.: 1 --> Item name = hamburger
Evaluating...
Starting Download...
Completed Image ====> 1.RedDot_Burger.jpg
Completed Image ====> 2.recipebest-hamburger.1024x1024.jpg
Completed Image ====> 3.20190725-delish-air-fryer-burger-ehg-vertical-2-1565299632.png
Completed Image ====> 4.1558017418187.jpeg
Completed Image ====> 5.4589_4k.jpg
Completed Image ====> 6.topimage-classic-hamburge

In [0]:
hamburger_paths = glob.glob("downloads/hamburger/*")
hot_dog_paths = glob.glob("downloads/hot-dog/*")

print('Imágenes de hamburguesas: ', len(hamburger_paths))
print('Imágenes de hot-dogs: ', len(hot_dog_paths))

Imágenes de hamburguesas:  10
Imágenes de hot-dogs:  93


# Flattening y agregando label

In [0]:
def flatten(imagePath, label):
    data = np.array([])
    for i in range(len(imagePath)):
        resized = cv2.resize(cv2.imread(imagePath[i]), dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
    
        data = np.append(data, np.append(resized, [label]))
    pixels = data.flatten().reshape(len(imagePath), 64*64*3+1)    

    return pixels

In [0]:
hamburger_data = flatten(hamburger_paths, 0)
hot_dog_data = flatten(hot_dog_paths, 1)

In [0]:
print(hamburger_data.shape)
print(hot_dog_data.shape)
print(hamburger_data[0])

(98, 12289)
(91, 12289)
[226. 214. 209. ... 237. 222.   0.]


# Juntamos los datos de las dos clases

In [0]:
x = []
label = []
for image in hamburger_data:
    x.append(image[:-1])
    label.append(image[-1])
    
for image in hot_dog_data:
    x.append(image[:-1])
    label.append(image[-1])

# Modelo

Ahora tenemos un dataset. En este dataset, la x son los training features. Para una imágen, tenemos 64*64*3 pixeles. Cada uno de estos números servirá al modelo para aprender a predecir si es una imagen de un hot dog o de una hamburguesa.

Utilizaremos la librería Scikit Learn para implementar el modelo. Scikit Learn es una librería de Python que tiene implementaciones de difeentes técnicas de Machine Learning como regresión linear, random forests, support vector machines y más.

En esta primera y simple versión, tu objetivo es implementar un clasificador. Deberás importar la librería, definir el modelo y sus hiperparámetros, y entrenarlo. Te recomiendo revisar los ejemplos en la documentación de Sklear.

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [26]:
# Importa las librerías que te interesen de sklearn
from sklearn import svm, metrics

# Define el modelo
classifier = svm.SVC(gamma=0.001)

# Usa el método .fit para entrenar el modelo.
classifier.fit(x, label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Métricas

Ejecuta las siguientes celdas para observar las métricas del modelo.

In [0]:
expected = label
predicted = classifier.predict(x)

In [28]:
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))

Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       142
         1.0       1.00      1.00      1.00        93

    accuracy                           1.00       235
   macro avg       1.00      1.00      1.00       235
weighted avg       1.00      1.00      1.00       235


